In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np

In [12]:
data_dir = r"G:\Numbers"

In [13]:
classes = os.listdir(data_dir)

In [14]:
image_list = []
label_list = []

In [15]:
for class_name in classes:
    class_path = os.path.join(data_dir, class_name)
    for img_name in os.listdir(class_path)[:]:  # Limit to the first 1200 images per class
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)

        if img is not None:
            img = cv2.resize(img, (64, 64))  # Resize the images to a common size
            image_list.append(img)
            label_list.append(int(class_name)) 

In [16]:
images = np.array(image_list)
labels = np.array(label_list)

In [17]:
labels_one_hot = to_categorical(labels - 1, num_classes=len(classes))  # Subtract 1 to make labels start from 0



In [18]:
X_train, X_test, y_train, y_test = train_test_split(images, labels_one_hot, test_size=0.2, random_state=42)


In [19]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.8)) 
model.add(layers.Dense(64, activation='relu',kernel_regularizer='l2'))
model.add(layers.Dense(len(classes), activation='softmax'))


In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [22]:
datagen.fit(X_train)

In [26]:
model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=3, validation_data=(X_test, y_test))

Epoch 1/3
270/270 [==============================] - 64s 237ms/step - loss: 0.9472 - accuracy: 0.8963 - val_loss: 0.6086 - val_accuracy: 1.0000
Epoch 2/3
270/270 [==============================] - 64s 239ms/step - loss: 0.7916 - accuracy: 0.9117 - val_loss: 1.0111 - val_accuracy: 0.8296
Epoch 3/3
270/270 [==============================] - 64s 239ms/step - loss: 0.7395 - accuracy: 0.9149 - val_loss: 0.7070 - val_accuracy: 0.9019


In [27]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")

68/68 [==============================] - 4s 61ms/step - loss: 0.7070 - accuracy: 0.9019

Test Accuracy: 90.19%
